In [1]:
from pyspark.sql import SparkSession
import time

sc = SparkSession \
    .builder \
    .appName("RDD query 1 execution") \
    .config("spark.executor.instances", "4") \
    .getOrCreate() \
    .sparkContext

# Load the CSV data
data_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'

df = spark.read.csv(data_path, header=True, inferSchema=True)

# Print the schema of the dataframe
df.printSchema()


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3474,application_1732639283265_3430,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- 

In [2]:
data2_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
df2 = spark.read.csv(data2_path,header = True, inferSchema = True)
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DR_NO: integer (nullable = true)
 |-- Date Rptd: string (nullable = true)
 |-- DATE OCC: string (nullable = true)
 |-- TIME OCC: integer (nullable = true)
 |-- AREA : integer (nullable = true)
 |-- AREA NAME: string (nullable = true)
 |-- Rpt Dist No: integer (nullable = true)
 |-- Part 1-2: integer (nullable = true)
 |-- Crm Cd: integer (nullable = true)
 |-- Crm Cd Desc: string (nullable = true)
 |-- Mocodes: string (nullable = true)
 |-- Vict Age: integer (nullable = true)
 |-- Vict Sex: string (nullable = true)
 |-- Vict Descent: string (nullable = true)
 |-- Premis Cd: integer (nullable = true)
 |-- Premis Desc: string (nullable = true)
 |-- Weapon Used Cd: integer (nullable = true)
 |-- Weapon Desc: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Status Desc: string (nullable = true)
 |-- Crm Cd 1: integer (nullable = true)
 |-- Crm Cd 2: integer (nullable = true)
 |-- Crm Cd 3: integer (nullable = true)
 |-- Crm Cd 4: integer (nullable = true)
 |-- 

In [3]:
# Number of rows and columns in df
print(f"df rows: {df.count()}, columns: {len(df.columns)}")

# Number of rows and columns in df2
print(f"df2 rows: {df2.count()}, columns: {len(df2.columns)}")

# Combined DataFrame
df_combined = df.union(df2)

# Number of rows and columns in df_combined
print(f"df_combined rows: {df_combined.count()}, columns: {len(df_combined.columns)}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df rows: 2123044, columns: 28
df2 rows: 990293, columns: 28
df_combined rows: 3113337, columns: 28

In [4]:
df_combined.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA ', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON']

## RDD - Sorted AGGRAVATED ASSAULT CRIMES BY AGE GROUP

In [5]:
import csv

# Λειτουργία για να διαβάσεις τα δεδομένα CSV και να τα επεξεργαστείς με csv.reader
def parse_csv(line):
    # Χρησιμοποιούμε το csv.reader για να επεξεργαστούμε σωστά την γραμμή
    return list(csv.reader([line]))[0]

# Διαβάζουμε τα δεδομένα από το S3 και τα περνάμε μέσω του csv.reader
data = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv") \
            .map(parse_csv)

data2 = sc.textFile("s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv") \
            .map(parse_csv)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Διαβάζουμε και φιλτράρουμε τα δύο RDDs
start_time = time.time()

# Διαβάζω και φιλτράρω τα δύο RDDs
data_filtered = data.filter(lambda x: x[11].isdigit() and 64 < int(x[11]) and 
                            (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                             x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                    .map(lambda x: [int(x[11]), [x[0], x[9]]])

data2_filtered = data2.filter(lambda x: x[11].isdigit() and 64 < int(x[11]) and 
                              (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                               x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                     .map(lambda x: [int(x[11]), [x[0], x[9]]])

# Ενώνω τα δύο RDDs
combined_data = data_filtered.union(data2_filtered)

# Ταξινομώ τα δεδομένα με βάση το key (ηλικία)
sorted_combined_data = combined_data.sortByKey(ascending=False)

# Εκτύπωση αποτελεσμάτων
print("Sorted by Elderly")
for item in sorted_combined_data.take(20):
    print(item)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sorted by Elderly
(120, ['200808494', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['170132132', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['171805899', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['180121778', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(99, ['181805391', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(99, ['180112253', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['180514402', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(99, ['101408873', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['210510500', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['210817606', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(99, ['210912586', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(99, ['210401098', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['221816955', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['160108922', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(99, ['160509846', 'ASSAULT WITH DEADLY WEAPON

In [7]:
# Διαβάζω και φιλτράρω τα δύο RDDs
data_filtered = data.filter(lambda x: x[11].isdigit() and 25 <= int(x[11]) <= 64 and 
                            (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                             x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                    .map(lambda x: [int(x[11]), [x[0], x[9]]])

data2_filtered = data2.filter(lambda x: x[11].isdigit() and 25 <= int(x[11]) <= 64 and 
                              (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                               x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                     .map(lambda x: [int(x[11]), [x[0], x[9]]])

# Ενώνω τα δύο RDDs
combined_data = data_filtered.union(data2_filtered)

# Ταξινομώ τα δεδομένα με βάση το key (ηλικία)
sorted_combined_data = combined_data.sortByKey(ascending=False)

# Εκτύπωση αποτελεσμάτων
print("Sorted by Adult")
for item in sorted_combined_data.take(20):
    print(item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sorted by Adult
(64, ['100115360', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100116279', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100117603', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100120120', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100120846', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100214001', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100311109', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100319816', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100410531', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100514181', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100716850', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100816337', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['100919279', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64, ['101208131', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(64,

In [8]:
# Διαβάζω και φιλτράρω τα δύο RDDs
data_filtered = data.filter(lambda x: x[11].isdigit() and 18 <= int(x[11]) <= 24 and 
                            (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                             x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                    .map(lambda x: [int(x[11]), [x[0], x[9]]])

data2_filtered = data2.filter(lambda x: x[11].isdigit() and 18 <= int(x[11]) <= 24 and 
                              (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                               x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                     .map(lambda x: [int(x[11]), [x[0], x[9]]])

# Ενώνω τα δύο RDDs
combined_data = data_filtered.union(data2_filtered)

# Ταξινομώ τα δεδομένα με βάση το key (ηλικία)
sorted_combined_data = combined_data.sortByKey(ascending=False)

# Εκτύπωση αποτελεσμάτων
print("Sorted by Youth Adult")
for item in sorted_combined_data.take(20):
    print(item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sorted by Youth Adult
(24, ['121900610', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['120625403', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['122100506', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['122110725', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['121300985', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['121816515', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['120216516', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['122020335', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['121906512', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(24, ['121901525', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['120115458', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['121906101', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['120621113', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['130605157', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(24, ['

In [9]:
# Φιλτράρισμα για τις συγκεκριμένες περιγραφές εγκλημάτων και την ηλικία
data_filtered = data.filter(lambda x: x[11].isdigit() and int(x[11]) < 18 and 
                            (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                             x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                    .map(lambda x: [int(x[11]), [x[0], x[9]]])

data2_filtered = data2.filter(lambda x: x[11].isdigit() and int(x[11]) < 18 and 
                              (x[9] == "ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT" or 
                               x[9] == "INTIMATE PARTNER - AGGRAVATED ASSAULT")) \
                     .map(lambda x: [int(x[11]), [x[0], x[9]]])

# Ενώνουμε τα δύο RDDs
combined_data = data_filtered.union(data2_filtered)

# Ταξινομούμε τα δεδομένα με βάση το key (ηλικία)
sorted_combined_data = combined_data.sortByKey(ascending=False)

# Εκτύπωση αποτελεσμάτων
print("Sorted by Children")
for item in sorted_combined_data.take(20):
    print(item)
# Stop timing and print out the execution duration
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sorted by Children
(17, ['200408755', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['201113788', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['201820292', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200705347', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200612219', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(17, ['200305246', 'INTIMATE PARTNER - AGGRAVATED ASSAULT'])
(17, ['200612388', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['201208407', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200904420', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200811596', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['201100618', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200212935', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200408928', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['200312712', 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT'])
(17, ['201500734', 